# This notebook will be used as a check mechanism for the workings used in calculating the IFRS 17 Figures for the acturial team. 

## Description 

* The purpose of this model is to group the EARe Non Life contracts into the 3 profitability groups, as prescribed under IFRS 17.
* This model acts as a parallel check to the automated grouping which takes place in Legerity, the IFRS 17 reporting engine. 



## Inputs 
1. Data used originates from a comprehensive contract register encompassing premiums, claims, commissions, profit commissions, and premium taxes allocated to each underwritten contract from 2001 to the present date.
    * The comprehensive contract register is updated on an ongoing basis
    * Contract Grouping - The register is shared by the Finance Team on a quarterly basis. 




## Outputs 
1. Column containing Profit ratio
2. Column containing Combined ratio
3. Column containing Profitability Grouping
4. Column containing Profitability Grouped Code



#### Things to keep in mind
* An expense loading (management expenses), is loaded over and above the contract specific expenses such as PC, premium Taxes and other charges, to determine the combined ratio.
* A Risk Adjustment percentage is loaded over and above the Claims, Commissions and Management expenses, as is required under IFRS 17
* Onerous Contracts - Any contract with a combined ratio greater than 100% are classified as Onerous contracts
* In Between Contracts - Any contract with a combined ratio between 90% and 100% will be classified as In Between Contracts
* Profitable Contracts - Any contract with a combined ratio of less than 90% will be classidied as a Profitable Contract


# IFRS17 None-Life Script

#### Importing the required libraries 

In [5]:
import warnings
warnings.filterwarnings("ignore")
from timeit import default_timer as timer #Recording what the runtime is
start = timer()

#Import libraries
import pandas as pd
import numpy as np
#To work with date inputs
from datetime import date, datetime
import tkinter as tk#for the base tinker functions
import tkinter.ttk as ttk# for themed tinker functions
from tkinter import filedialog #To read in files 
from decimal import Decimal




# Reading in our main input file 

In [6]:
start2 = timer()

path2 = './Data/PC TRANSACTION DATA Q3.csv'

path2_ = './Data/NL Actuarial Input_Sep_Mod.xlsx'

df = pd.read_csv(path2)
act_input = pd.read_excel(path2_)

df.info()
end = timer()
duration = end - start2
print('It takes this many seconds to read in our files: ',duration)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 121076 entries, 0 to 121075
Data columns (total 14 columns):
 #   Column             Non-Null Count   Dtype  
---  ------             --------------   -----  
 0   CONTRACT_NO        121076 non-null  object 
 1   UNDERWRITING_YEAR  121076 non-null  int64  
 2   FINANCIAL_YEAR     121075 non-null  float64
 3   FINANCIAL_QUARTER  121075 non-null  object 
 4   TYPE_OF_BUSINESS   121076 non-null  object 
 5   LINE_OF_BUSINESS   121076 non-null  object 
 6   POLICY_ID          121076 non-null  object 
 7   BOOKED_PREMIUM     121076 non-null  float64
 8   COMM_BROKERAGE     121076 non-null  float64
 9   ACQUISITIONCOSTS   121076 non-null  float64
 10  CLAIM_PAID         121076 non-null  float64
 11  EXPENSE_AMOUNT     121076 non-null  float64
 12  EXPENSE            121076 non-null  float64
 13  EXPENSE_LOADING    121076 non-null  float64
dtypes: float64(8), int64(1), object(5)
memory usage: 12.9+ MB
It takes this many seconds to read in our 

In [7]:
#How does our data look like 
df.head(5)

,CONTRACT_NO,UNDERWRITING_YEAR,FINANCIAL_YEAR,FINANCIAL_QUARTER,TYPE_OF_BUSINESS,LINE_OF_BUSINESS,POLICY_ID,BOOKED_PREMIUM,COMM_BROKERAGE,ACQUISITIONCOSTS,CLAIM_PAID,EXPENSE_AMOUNT,EXPENSE,EXPENSE_LOADING
0,91,2008,2008.0,31/12/2008,PROPORTIONAL,FIRE,FIRE_2008,448361.00,0.00,0.00,0.00,-17655.0,3.937675,5.5414
1,91,2011,2011.0,30/09/2011,PROPORTIONAL,FIRE,FIRE_2011,1688514.00,-629914.00,-629914.00,-2155509.00,-15053.0,0.891494,5.5414
2,91,2015,2015.0,30/09/2015,PROPORTIONAL,FIRE,FIRE_2015,2207111.00,-817207.00,-817207.00,-599126.00,-21128.0,0.957269,5.5414
3,91,2015,2016.0,30/06/2016,PROPORTIONAL,FIRE,FIRE_2015,0.00,0.00,0.00,-1507410.75,0.0,0.000000,5.5414
4,182,2007,2007.0,30/09/2007,PROPORTIONAL,FIRE,FIRE_2007,14039958.26,-4874103.52,-4874103.52,3449222.94,0.0,0.000000,5.5414


### We dont need all of the columns so the first step is creating a slice limited to what well use/need

### Filter the info passed on the calculation sections, this allows users to choose upto when they would want to 'report by'

Time to valiadate the input and convert it into an date variable

In [8]:
from datetime import datetime

print("Please select the reporting date that you want to calculate by")
year = int(input('Enter the year as a whole number(e.g 2022): '))
month = int(input('Enter the month as a number with no leading 0s(e.g 4): '))
day = int(input('Enter the day as a number with no leading 0s(e.g 31): '))



# To validate user input
def validate_date():
  try:
    reporting_date = datetime(int(year), int(month), int(day))
  except ValueError:
    print("Invalid date")
    return False

  if 2004 <= reporting_date.year <= 2023:  
    if reporting_date.month in [4, 7, 9, 12]:
      return True
    else:
      print("Invalid month")
  else: 
    print("Year in Range")

  return False

#Run check
validate_date()

#This is our date filter now
reporting_date = date(year, month, day)
print(reporting_date)


Please select the reporting date that you want to calculate by
Invalid year
2024-09-30


Lets get user input on what the expense loading and risk adjustment should be

In [9]:
print('Please provide the following: ')
# expense_loading = float(input('Enter the expense loading as a decimal: '))
sensitivity_loading = 0.02
print('\n\n\n Begin Run')

Please provide the following: 



 Begin Run


What range of dates does our data span

In [10]:

df['FINANCIAL_QUARTER'] = pd.to_datetime(df['FINANCIAL_QUARTER'])

df['FINANCIAL_QUARTER']

0        2008-12-31
1        2011-09-30
2        2015-09-30
3        2016-06-30
4        2007-09-30
            ...    
121071   2024-03-31
121072   2023-12-31
121073   2024-09-30
121074   2024-03-31
121075   2024-06-30
Name: FINANCIAL_QUARTER, Length: 121076, dtype: datetime64[ns]

In [11]:
sorted(df['FINANCIAL_QUARTER'].value_counts())
#We now filter our data set using the dates we have 
df['FINANCIAL_QUARTER'] = df['FINANCIAL_QUARTER'].dt.date

df = df.loc[df['FINANCIAL_QUARTER'] <= reporting_date]#By the FINANCIAL quaater
df.info()#Taking a look at what we are left with 

<class 'pandas.core.frame.DataFrame'>
Index: 120850 entries, 0 to 121075
Data columns (total 14 columns):
 #   Column             Non-Null Count   Dtype  
---  ------             --------------   -----  
 0   CONTRACT_NO        120850 non-null  object 
 1   UNDERWRITING_YEAR  120850 non-null  int64  
 2   FINANCIAL_YEAR     120850 non-null  float64
 3   FINANCIAL_QUARTER  120850 non-null  object 
 4   TYPE_OF_BUSINESS   120850 non-null  object 
 5   LINE_OF_BUSINESS   120850 non-null  object 
 6   POLICY_ID          120850 non-null  object 
 7   BOOKED_PREMIUM     120850 non-null  float64
 8   COMM_BROKERAGE     120850 non-null  float64
 9   ACQUISITIONCOSTS   120850 non-null  float64
 10  CLAIM_PAID         120850 non-null  float64
 11  EXPENSE_AMOUNT     120850 non-null  float64
 12  EXPENSE            120850 non-null  float64
 13  EXPENSE_LOADING    120850 non-null  float64
dtypes: float64(8), int64(1), object(5)
memory usage: 13.8+ MB


Next we create a slice of the columns we will utilize

In [12]:
#Creating a slice 
# Expense amount is just expenses and comm-brokrage is already included in the ACQUISITION COSTS
data = df[['CONTRACT_NO','UNDERWRITING_YEAR', 'LINE_OF_BUSINESS', 
           'POLICY_ID','BOOKED_PREMIUM','CLAIM_PAID','ACQUISITIONCOSTS','EXPENSE_AMOUNT']]
data.info()

<class 'pandas.core.frame.DataFrame'>
Index: 120850 entries, 0 to 121075
Data columns (total 8 columns):
 #   Column             Non-Null Count   Dtype  
---  ------             --------------   -----  
 0   CONTRACT_NO        120850 non-null  object 
 1   UNDERWRITING_YEAR  120850 non-null  int64  
 2   LINE_OF_BUSINESS   120850 non-null  object 
 3   POLICY_ID          120850 non-null  object 
 4   BOOKED_PREMIUM     120850 non-null  float64
 5   CLAIM_PAID         120850 non-null  float64
 6   ACQUISITIONCOSTS   120850 non-null  float64
 7   EXPENSE_AMOUNT     120850 non-null  float64
dtypes: float64(4), int64(1), object(3)
memory usage: 8.3+ MB


In [13]:
#Converting some columns into strings 
data['CONTRACT_NO'] = data['CONTRACT_NO'].astype('string')
data['LINE_OF_BUSINESS'] = data['LINE_OF_BUSINESS'].astype('string')
data['POLICY_ID'] = data['POLICY_ID'].astype('string')
#Just to confirm we have what we need
print("Lets confrim that there are no missing sections ",data.isna().sum())

Lets confrim that there are no missing sections  CONTRACT_NO          0
UNDERWRITING_YEAR    0
LINE_OF_BUSINESS     0
POLICY_ID            0
BOOKED_PREMIUM       0
CLAIM_PAID           0
ACQUISITIONCOSTS     0
EXPENSE_AMOUNT       0
dtype: int64


## 1. Calculating the total costs

In [14]:
#The filter for risk adjustment

# This is the round off of the Standard Deviation of the Loss Ratio for the last 6 years 
#sensitivity_loading = 0.02
def assign_values(row):
    if row['LINE_OF_BUSINESS'] == 'FIRE':
        return 0.059
    elif row['LINE_OF_BUSINESS'] == 'MISCELLANEOUS':
        return 0.112
    elif row['LINE_OF_BUSINESS'] == 'MOTOR':
        return 0.066
    elif row['LINE_OF_BUSINESS'] == 'MARINE':
        return 0.084
    elif row['LINE_OF_BUSINESS'] == 'ENGINEERING':
        return 0.059
    elif row['LINE_OF_BUSINESS'] == 'WHOLE_ACCOUNT':
        return 0.02
    elif row['LINE_OF_BUSINESS'] == 'LIABILITY':
        return 0.031
    elif row['LINE_OF_BUSINESS'] == 'MEDICAL':
        return 0.026
    elif row['LINE_OF_BUSINESS'] == 'PERSONAL_ACCIDENT':
        return 0.031
    elif row['LINE_OF_BUSINESS'] == 'AVIATION':
        return 0.031
    elif row['LINE_OF_BUSINESS'] == 'THEFT':
        return 0
    elif row['LINE_OF_BUSINESS'] == 'WORKMENS_COMPENSATION':
        return 0

data['Risk_Adjustment'] = data.apply(assign_values, axis=1)


In [15]:
# Creating a column for risk adjustment 

data['CLAIMS_PAID_ADJ'] = data['CLAIM_PAID'] * ( data['Risk_Adjustment'] + 1) 
#*********
start3 = timer()
#Making the total costs column
data['TOTAL_COSTS'] = data['EXPENSE_AMOUNT'] + data['ACQUISITIONCOSTS']  + data['CLAIM_PAID']
#data = data.drop(['EXPENSE_AMOUNT','ACQUISITIONCOSTS'],axis=1)
#***********
#Turning the costs +ve due to how they were entered into the spreadsheet
data['TOTAL_COSTS'] = data['TOTAL_COSTS'] * -1
#***************
data.head()


#Lets attempt to replace the missing premiums with NaN early on
data['BOOKED_PREMIUM'].replace(0, np.NaN, inplace=True)
data['BOOKED_PREMIUM'].value_counts()



BOOKED_PREMIUM
1.00         54
45000.00     31
135000.00    31
90000.00     29
108000.00    27
             ..
466446.00     1
351456.00     1
101982.00     1
159994.00     1
2873.42       1
Name: count, Length: 77473, dtype: int64

## 2. Calculating the combined ratio

In [16]:
#Getting the grouped premium
worked_prem = data.groupby(['LINE_OF_BUSINESS','CONTRACT_NO'], as_index=False, group_keys=True)['BOOKED_PREMIUM'].sum()
print("Premium size: ",worked_prem.shape)

#Getting the grouped cost
worked_cost = data.groupby(['LINE_OF_BUSINESS','CONTRACT_NO'], as_index=False, group_keys=True)['TOTAL_COSTS'].sum()
worked_cost['GROUPED_PREMIUM'] = worked_prem['BOOKED_PREMIUM']
worked_cost.rename(columns = {'TOTAL_COSTS':'GROUPED_COSTS'}, inplace = True)
print("Cost size: ",worked_cost.shape)

#Merging the 2 frames together
grouped_transactions = pd.merge(data,worked_cost, on=['LINE_OF_BUSINESS','CONTRACT_NO'], how='left')
print("Together size: ",grouped_transactions.shape)

print('Lets visualize how the frame looks like now')
grouped_transactions.head(20)

Premium size:  (7780, 3)
Cost size:  (7780, 4)
Together size:  (120850, 13)
Lets visualize how the frame looks like now


,CONTRACT_NO,UNDERWRITING_YEAR,LINE_OF_BUSINESS,POLICY_ID,BOOKED_PREMIUM,CLAIM_PAID,ACQUISITIONCOSTS,EXPENSE_AMOUNT,Risk_Adjustment,CLAIMS_PAID_ADJ,TOTAL_COSTS,GROUPED_COSTS,GROUPED_PREMIUM
0,91,2008,FIRE,FIRE_2008,448361.00,0.00,0.00,-17655.00,0.059,0.000000e+00,17655.00,82133245.69,63325746.83
1,91,2011,FIRE,FIRE_2011,1688514.00,-2155509.00,-629914.00,-15053.00,0.059,-2.282684e+06,2800476.00,82133245.69,63325746.83
2,91,2015,FIRE,FIRE_2015,2207111.00,-599126.00,-817207.00,-21128.00,0.059,-6.344744e+05,1437461.00,82133245.69,63325746.83
3,91,2015,FIRE,FIRE_2015,NaN,-1507410.75,0.00,0.00,0.059,-1.596348e+06,1507410.75,82133245.69,63325746.83
4,182,2007,FIRE,FIRE_2007,14039958.26,3449222.94,-4874103.52,0.00,0.059,3.652727e+06,1424880.58,61945052.06,97802337.74
5,182,2007,FIRE,FIRE_2007,-5718370.84,-11381002.36,2037513.52,0.00,0.059,-1.205248e+07,9343488.84,61945052.06,97802337.74
6,273,2006,FIRE,FIRE_2006,132861.00,-987.75,-49822.88,-1992.90,0.059,-1.046027e+03,52803.53,72452929.81,93633037.36
7,273,2009,FIRE,FIRE_2009,226271.85,-26181.00,-84852.00,-3394.05,0.059,-2.772568e+04,114427.05,72452929.81,93633037.36
8,273,2012,FIRE,FIRE_2012,902388.30,-665920.50,-338395.66,-13535.70,0.059,-7.052098e+05,1017851.86,72452929.81,93633037.36
9,273,2012,FIRE,FIRE_2012,2475101.10,4017110.25,0.00,0.00,0.059,4.254120e+06,-4017110.25,72452929.81,93633037.36


## 3. Calculating the combined loss ratio by Contract No

In [17]:
grouped_transactions['CONTRACT_COMBINED_RATIO'] = grouped_transactions['GROUPED_COSTS'] / grouped_transactions['GROUPED_PREMIUM']
#Lets change the LOB column into a string and
# use it to create the profit code
x = grouped_transactions['CONTRACT_COMBINED_RATIO'].isna().sum()
print('There are ' + str(x) + ' problematic CONTRACT_COMBINED_RATIO rows null  in the table, this is due to negative or impossible values from calculations')

There are 594 problematic CONTRACT_COMBINED_RATIO rows null  in the table, this is due to negative or impossible values from calculations


In [18]:
grouped_transactions.head(10)

,CONTRACT_NO,UNDERWRITING_YEAR,LINE_OF_BUSINESS,POLICY_ID,BOOKED_PREMIUM,CLAIM_PAID,ACQUISITIONCOSTS,EXPENSE_AMOUNT,Risk_Adjustment,CLAIMS_PAID_ADJ,TOTAL_COSTS,GROUPED_COSTS,GROUPED_PREMIUM,CONTRACT_COMBINED_RATIO
0,91,2008,FIRE,FIRE_2008,448361.00,0.00,0.00,-17655.00,0.059,0.000000e+00,17655.00,82133245.69,63325746.83,1.296996
1,91,2011,FIRE,FIRE_2011,1688514.00,-2155509.00,-629914.00,-15053.00,0.059,-2.282684e+06,2800476.00,82133245.69,63325746.83,1.296996
2,91,2015,FIRE,FIRE_2015,2207111.00,-599126.00,-817207.00,-21128.00,0.059,-6.344744e+05,1437461.00,82133245.69,63325746.83,1.296996
3,91,2015,FIRE,FIRE_2015,NaN,-1507410.75,0.00,0.00,0.059,-1.596348e+06,1507410.75,82133245.69,63325746.83,1.296996
4,182,2007,FIRE,FIRE_2007,14039958.26,3449222.94,-4874103.52,0.00,0.059,3.652727e+06,1424880.58,61945052.06,97802337.74,0.633370
5,182,2007,FIRE,FIRE_2007,-5718370.84,-11381002.36,2037513.52,0.00,0.059,-1.205248e+07,9343488.84,61945052.06,97802337.74,0.633370
6,273,2006,FIRE,FIRE_2006,132861.00,-987.75,-49822.88,-1992.90,0.059,-1.046027e+03,52803.53,72452929.81,93633037.36,0.773797
7,273,2009,FIRE,FIRE_2009,226271.85,-26181.00,-84852.00,-3394.05,0.059,-2.772568e+04,114427.05,72452929.81,93633037.36,0.773797
8,273,2012,FIRE,FIRE_2012,902388.30,-665920.50,-338395.66,-13535.70,0.059,-7.052098e+05,1017851.86,72452929.81,93633037.36,0.773797
9,273,2012,FIRE,FIRE_2012,2475101.10,4017110.25,0.00,0.00,0.059,4.254120e+06,-4017110.25,72452929.81,93633037.36,0.773797


### As there are quite a number of div/0 cases, the standard is to replace these by the LOB average combined ratio(i.e fires combined ratio)

In [19]:
working_frame = grouped_transactions.copy()
working_frame.rename(columns ={'BOOKED_PREMIUM':'PREMIUMS'}, inplace = True)
working_frame

,CONTRACT_NO,UNDERWRITING_YEAR,LINE_OF_BUSINESS,POLICY_ID,PREMIUMS,CLAIM_PAID,ACQUISITIONCOSTS,EXPENSE_AMOUNT,Risk_Adjustment,CLAIMS_PAID_ADJ,TOTAL_COSTS,GROUPED_COSTS,GROUPED_PREMIUM,CONTRACT_COMBINED_RATIO
0,91,2008,FIRE,FIRE_2008,448361.00,0.00,0.00,-17655.00,0.059,0.000000e+00,17655.00,82133245.69,63325746.83,1.296996
1,91,2011,FIRE,FIRE_2011,1688514.00,-2155509.00,-629914.00,-15053.00,0.059,-2.282684e+06,2800476.00,82133245.69,63325746.83,1.296996
2,91,2015,FIRE,FIRE_2015,2207111.00,-599126.00,-817207.00,-21128.00,0.059,-6.344744e+05,1437461.00,82133245.69,63325746.83,1.296996
3,91,2015,FIRE,FIRE_2015,NaN,-1507410.75,0.00,0.00,0.059,-1.596348e+06,1507410.75,82133245.69,63325746.83,1.296996
4,182,2007,FIRE,FIRE_2007,14039958.26,3449222.94,-4874103.52,0.00,0.059,3.652727e+06,1424880.58,61945052.06,97802337.74,0.633370
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
120845,PTTY1911,2023,LIABILITY,LIABILITY_2023,18680.96,0.00,-5137.26,-10333.37,0.031,0.000000e+00,15470.63,31718.39,77763.67,0.407882
120846,PTTY1921,2023,MISCELLANEOUS,MISCELLANEOUS_2023,NaN,0.00,0.00,0.00,0.112,0.000000e+00,-0.00,0.00,0.00,NaN
120847,PTTY1921,2023,MISCELLANEOUS,MISCELLANEOUS_2023,NaN,0.00,0.00,0.00,0.112,0.000000e+00,-0.00,0.00,0.00,NaN
120848,PTTY2130,2023,MARINE,MARINE_2023,22586.85,0.00,-6211.38,-225.87,0.084,0.000000e+00,6437.25,6437.25,22586.85,0.285000


In [20]:

#print("There are "+str(ratios_check))
#Replaces infinte values with the na values
def repalceWithNa(df):
  return df.replace([np.inf, -np.inf,np.nan], np.nan, inplace=True)
  
repalceWithNa(working_frame)


## We'll do it by line of business

In [21]:
#Getting the grouped premium
worked_prem = working_frame.groupby(['LINE_OF_BUSINESS'], as_index=False, group_keys=True)['PREMIUMS'].sum()
print("Premium size: ",worked_prem.shape)

#Getting the grouped cost
worked_cost = working_frame.groupby(['LINE_OF_BUSINESS'], as_index=False, group_keys=True)['TOTAL_COSTS'].sum()
worked_cost['Group_premium_LOB'] = worked_prem['PREMIUMS']
worked_cost.rename(columns = {'TOTAL_COSTS':'Grouped_Costs_LOB'}, inplace = True)
print("Cost size: ",worked_cost.shape)

#Merging the 2 frames together
grouped_transactions = pd.merge(working_frame,worked_cost, on=['LINE_OF_BUSINESS'], how='left')
print("Together size: ",grouped_transactions.shape)

print('Lets visualize how the frame looks like now')
grouped_transactions.head(20)

Premium size:  (12, 2)
Cost size:  (12, 3)
Together size:  (120850, 16)
Lets visualize how the frame looks like now


,CONTRACT_NO,UNDERWRITING_YEAR,LINE_OF_BUSINESS,POLICY_ID,PREMIUMS,CLAIM_PAID,ACQUISITIONCOSTS,EXPENSE_AMOUNT,Risk_Adjustment,CLAIMS_PAID_ADJ,TOTAL_COSTS,GROUPED_COSTS,GROUPED_PREMIUM,CONTRACT_COMBINED_RATIO,Grouped_Costs_LOB,Group_premium_LOB
0,91,2008,FIRE,FIRE_2008,448361.00,0.00,0.00,-17655.00,0.059,0.000000e+00,17655.00,82133245.69,63325746.83,1.296996,1.617638e+10,1.812196e+10
1,91,2011,FIRE,FIRE_2011,1688514.00,-2155509.00,-629914.00,-15053.00,0.059,-2.282684e+06,2800476.00,82133245.69,63325746.83,1.296996,1.617638e+10,1.812196e+10
2,91,2015,FIRE,FIRE_2015,2207111.00,-599126.00,-817207.00,-21128.00,0.059,-6.344744e+05,1437461.00,82133245.69,63325746.83,1.296996,1.617638e+10,1.812196e+10
3,91,2015,FIRE,FIRE_2015,NaN,-1507410.75,0.00,0.00,0.059,-1.596348e+06,1507410.75,82133245.69,63325746.83,1.296996,1.617638e+10,1.812196e+10
4,182,2007,FIRE,FIRE_2007,14039958.26,3449222.94,-4874103.52,0.00,0.059,3.652727e+06,1424880.58,61945052.06,97802337.74,0.633370,1.617638e+10,1.812196e+10
5,182,2007,FIRE,FIRE_2007,-5718370.84,-11381002.36,2037513.52,0.00,0.059,-1.205248e+07,9343488.84,61945052.06,97802337.74,0.633370,1.617638e+10,1.812196e+10
6,273,2006,FIRE,FIRE_2006,132861.00,-987.75,-49822.88,-1992.90,0.059,-1.046027e+03,52803.53,72452929.81,93633037.36,0.773797,1.617638e+10,1.812196e+10
7,273,2009,FIRE,FIRE_2009,226271.85,-26181.00,-84852.00,-3394.05,0.059,-2.772568e+04,114427.05,72452929.81,93633037.36,0.773797,1.617638e+10,1.812196e+10
8,273,2012,FIRE,FIRE_2012,902388.30,-665920.50,-338395.66,-13535.70,0.059,-7.052098e+05,1017851.86,72452929.81,93633037.36,0.773797,1.617638e+10,1.812196e+10
9,273,2012,FIRE,FIRE_2012,2475101.10,4017110.25,0.00,0.00,0.059,4.254120e+06,-4017110.25,72452929.81,93633037.36,0.773797,1.617638e+10,1.812196e+10


In [22]:
repalceWithNa(grouped_transactions['CONTRACT_COMBINED_RATIO'])
# replace field that's entirely space (or empty) with NaN
grouped_transactions['CONTRACT_COMBINED_RATIO'].replace(r'^\s*$', np.nan, regex=True)
#Fill all empty columns with 
#This creates the subset of the df based on the lob
def select_lob(frame, lob):
  return frame[frame['LINE_OF_BUSINESS'] == lob]

#Replaces infinte values with the avg ratio
def replace_infinite(df, avg_ratio):
  return df.replace(np.nan, avg_ratio, inplace=True)

#Creating the list of LOB's
lob = grouped_transactions['LINE_OF_BUSINESS'].unique().tolist()

#Calculates the average combined ratio for this subset 
def calcuCombinedRatioLOB(datset):
  return datset['Grouped_Costs_LOB'] / datset['Group_premium_LOB']

def replaceBlank(df,replacer):
  return df.fillna(replacer, inplace=True)
    
def replaceNAN(df, space):
  return df.replace('', space, inplace=True)




for item in lob:
    
  indices = grouped_transactions['LINE_OF_BUSINESS'] == item
    
  df = grouped_transactions.loc[indices]
    
  ratio = calcuCombinedRatioLOB(df)
  
  replaceNAN(df, ratio)
  replaceBlank(df,ratio)
  replace_infinite(df, ratio)  # Replaces infinite values directly in the subset DataFrame
    
    
  #print(item, ratio)
  # Update the original DataFrame with the modified subset DataFrame
 # Update the original DataFrame with the modified subset DataFrame
  grouped_transactions.loc[indices] = df

In [23]:
#Stripping the leading zeros from the CONTRACT_NO column
grouped_transactions['CONTRACT_NO'] = grouped_transactions['CONTRACT_NO'].str.lstrip('0')


def fill_blank_cells_with_division(df, column_to_check, column_dividend, column_divisor):
    # Identify rows with blank cells in the specified column
    blank_cells = df[column_to_check].isnull() | (df[column_to_check] == '')

    # Perform division and fill in the blank cells with the result
    df.loc[blank_cells, column_to_check] = df.loc[blank_cells, column_dividend] / df.loc[blank_cells, column_divisor]

    return df

df = grouped_transactions
# Specify columns for division and column to check for blank cells
column_to_check = 'CONTRACT_COMBINED_RATIO'
column_dividend = 'Grouped_Costs_LOB'
column_divisor = 'Group_premium_LOB'

# Fill blank cells in column_to_check with division of values from column_dividend and column_divisor
result_df = fill_blank_cells_with_division(df, column_to_check, column_dividend, column_divisor)

print(result_df)


       CONTRACT_NO  UNDERWRITING_YEAR LINE_OF_BUSINESS           POLICY_ID  \
0               91               2008             FIRE           FIRE_2008   
1               91               2011             FIRE           FIRE_2011   
2               91               2015             FIRE           FIRE_2015   
3               91               2015             FIRE           FIRE_2015   
4              182               2007             FIRE           FIRE_2007   
...            ...                ...              ...                 ...   
120845    PTTY1911               2023        LIABILITY      LIABILITY_2023   
120846    PTTY1921               2023    MISCELLANEOUS  MISCELLANEOUS_2023   
120847    PTTY1921               2023    MISCELLANEOUS  MISCELLANEOUS_2023   
120848    PTTY2130               2023           MARINE         MARINE_2023   
120849    PTTY2144               2023             FIRE           FIRE_2023   

           PREMIUMS  CLAIM_PAID  ACQUISITIONCOSTS  EXPENSE_AMOU

In [24]:
# #We then add an expense loading to our Combine ratio
expense_loading = 0.055414
result_df['CONTRACT_COMBINED_RATIO_Adj'] = result_df['CONTRACT_COMBINED_RATIO'] + expense_loading
# 
# #expense_loading = 0.05
# #sAVE TO EXCEL
# #result_df.to_excel('Sensitivity_Check_8.xlsx')
# #At this point we can make a copy of our dataframe to export into excel for checks. Any subsequent chnages will be done to the copy
# worked_LOB_Frame = working_frame.copy()

# #for grouped combined ratios
# #working_frame.to_excel('Combined_Ratio.xlsx')

## 4. Calculating the profit code groupings

In [25]:
#Using the combined ratio inclusive of the expense ratio
upper_bound = 1.0
lower_bound = 0.9

conditions = [
    (result_df['CONTRACT_COMBINED_RATIO_Adj'] >= upper_bound),
    (result_df['CONTRACT_COMBINED_RATIO_Adj'] < upper_bound) & (result_df['CONTRACT_COMBINED_RATIO_Adj'] >= lower_bound),  
    (result_df['CONTRACT_COMBINED_RATIO_Adj'] < lower_bound)
]
v_codes = ['O','U','P']

result_df['Profit_Code'] = np.select(conditions,v_codes)
result_df.head()


# n = result_df['Profit_Code'].unique()#Just to confirm 
  
# print("No.of.unique values in each column :\n",
#       n)

# #We'll then export the contract profit codes into excel
# #worked_LOB_Frame.to_excel('Contract_Profit_Codes.xlsx')

# result_df['Profit_Code'].value_counts()
# result_df.to_excel('Group_Check_Diff_RA.xlsx')

,CONTRACT_NO,UNDERWRITING_YEAR,LINE_OF_BUSINESS,POLICY_ID,PREMIUMS,CLAIM_PAID,ACQUISITIONCOSTS,EXPENSE_AMOUNT,Risk_Adjustment,CLAIMS_PAID_ADJ,TOTAL_COSTS,GROUPED_COSTS,GROUPED_PREMIUM,CONTRACT_COMBINED_RATIO,Grouped_Costs_LOB,Group_premium_LOB,CONTRACT_COMBINED_RATIO_Adj,Profit_Code
0,91,2008,FIRE,FIRE_2008,448361.00,0.00,0.00,-17655.0,0.059,0.000000e+00,17655.00,82133245.69,63325746.83,1.296996,1.617638e+10,1.812196e+10,1.352410,O
1,91,2011,FIRE,FIRE_2011,1688514.00,-2155509.00,-629914.00,-15053.0,0.059,-2.282684e+06,2800476.00,82133245.69,63325746.83,1.296996,1.617638e+10,1.812196e+10,1.352410,O
2,91,2015,FIRE,FIRE_2015,2207111.00,-599126.00,-817207.00,-21128.0,0.059,-6.344744e+05,1437461.00,82133245.69,63325746.83,1.296996,1.617638e+10,1.812196e+10,1.352410,O
3,91,2015,FIRE,FIRE_2015,NaN,-1507410.75,0.00,0.0,0.059,-1.596348e+06,1507410.75,82133245.69,63325746.83,1.296996,1.617638e+10,1.812196e+10,1.352410,O
4,182,2007,FIRE,FIRE_2007,14039958.26,3449222.94,-4874103.52,0.0,0.059,3.652727e+06,1424880.58,61945052.06,97802337.74,0.633370,1.617638e+10,1.812196e+10,0.688784,P


# Here is where we add the sensitivity checks on the profitable contracts

In [26]:
#The filter for risk adjustment

# This is the round off of the Standard Deviation of the Loss Ratio for the last 6 years 
sensitivity_loading = 0.02
worked_LOB_Frame = result_df.copy()
# Define the condition
condition = worked_LOB_Frame['Profit_Code'] != 'O'

# Apply changes to specific rows based on the condition
#Create a new column based on the condition
worked_LOB_Frame.loc[condition, 'Adjusted_CRatio'] = worked_LOB_Frame.loc[condition, 'CONTRACT_COMBINED_RATIO_Adj'] + sensitivity_loading # For rows where condition is True, set 'C' to values from column 'B' multiplied by 3

#create an additional column to see if any of the edited ones have passed the threshold into 'U'
worked_LOB_Frame['Profit_Code_Mod'] = np.select(conditions,v_codes)


conditions = [
    (worked_LOB_Frame['Adjusted_CRatio'] >= lower_bound),  
    (worked_LOB_Frame['Adjusted_CRatio'] < lower_bound)
]

values = ['Inbetween','Profitable']
v_codes = ['U','P']

worked_LOB_Frame['Profit_Code_Mod'] = np.select(conditions,v_codes)

# We can add a check to see which rows/contracts exactly have moved to different groupings
dissimilar_count = len(worked_LOB_Frame[worked_LOB_Frame['Profit_Code_Mod'] != worked_LOB_Frame['Profit_Code']])

print(f"Number of Profit codes that have changed values: {dissimilar_count}")

Number of Profit codes that have changed values: 29715


In [27]:
worked_LOB_Frame['Adjusted_CRatio'].fillna(worked_LOB_Frame['CONTRACT_COMBINED_RATIO_Adj'], inplace=True)
worked_LOB_Frame['Profit_Code_Mod'].replace('0', 'O', inplace=True)
worked_LOB_Frame

# worked_LOB_Frame.to_excel('Check_27th_Correct.xlsx')

,CONTRACT_NO,UNDERWRITING_YEAR,LINE_OF_BUSINESS,POLICY_ID,PREMIUMS,CLAIM_PAID,ACQUISITIONCOSTS,EXPENSE_AMOUNT,Risk_Adjustment,CLAIMS_PAID_ADJ,TOTAL_COSTS,GROUPED_COSTS,GROUPED_PREMIUM,CONTRACT_COMBINED_RATIO,Grouped_Costs_LOB,Group_premium_LOB,CONTRACT_COMBINED_RATIO_Adj,Profit_Code,Adjusted_CRatio,Profit_Code_Mod
0,91,2008,FIRE,FIRE_2008,448361.00,0.00,0.00,-17655.00,0.059,0.000000e+00,17655.00,82133245.69,63325746.83,1.296996,1.617638e+10,1.812196e+10,1.352410,O,1.352410,O
1,91,2011,FIRE,FIRE_2011,1688514.00,-2155509.00,-629914.00,-15053.00,0.059,-2.282684e+06,2800476.00,82133245.69,63325746.83,1.296996,1.617638e+10,1.812196e+10,1.352410,O,1.352410,O
2,91,2015,FIRE,FIRE_2015,2207111.00,-599126.00,-817207.00,-21128.00,0.059,-6.344744e+05,1437461.00,82133245.69,63325746.83,1.296996,1.617638e+10,1.812196e+10,1.352410,O,1.352410,O
3,91,2015,FIRE,FIRE_2015,NaN,-1507410.75,0.00,0.00,0.059,-1.596348e+06,1507410.75,82133245.69,63325746.83,1.296996,1.617638e+10,1.812196e+10,1.352410,O,1.352410,O
4,182,2007,FIRE,FIRE_2007,14039958.26,3449222.94,-4874103.52,0.00,0.059,3.652727e+06,1424880.58,61945052.06,97802337.74,0.633370,1.617638e+10,1.812196e+10,0.688784,P,0.708784,P
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
120845,PTTY1911,2023,LIABILITY,LIABILITY_2023,18680.96,0.00,-5137.26,-10333.37,0.031,0.000000e+00,15470.63,31718.39,77763.67,0.407882,1.094737e+08,2.386842e+08,0.463296,P,0.483296,P
120846,PTTY1921,2023,MISCELLANEOUS,MISCELLANEOUS_2023,NaN,0.00,0.00,0.00,0.112,0.000000e+00,-0.00,0.00,0.00,0.722122,4.513083e+09,6.249749e+09,0.777536,P,0.797536,P
120847,PTTY1921,2023,MISCELLANEOUS,MISCELLANEOUS_2023,NaN,0.00,0.00,0.00,0.112,0.000000e+00,-0.00,0.00,0.00,0.722122,4.513083e+09,6.249749e+09,0.777536,P,0.797536,P
120848,PTTY2130,2023,MARINE,MARINE_2023,22586.85,0.00,-6211.38,-225.87,0.084,0.000000e+00,6437.25,6437.25,22586.85,0.285000,2.411435e+09,3.033644e+09,0.340414,P,0.360414,P


## Creating the LOB_UY_CODE Grouping

In [28]:
worked_LOB_Frame['LOB_UY_GROUP'] = worked_LOB_Frame['POLICY_ID'] + '_' + worked_LOB_Frame['Profit_Code']

# Creating a new column for LOB_PC
worked_LOB_Frame.drop(['GROUPED_COSTS','GROUPED_PREMIUM','Grouped_Costs_LOB','Group_premium_LOB','CONTRACT_COMBINED_RATIO'], axis=1,inplace=True)
worked_LOB_Frame

,CONTRACT_NO,UNDERWRITING_YEAR,LINE_OF_BUSINESS,POLICY_ID,PREMIUMS,CLAIM_PAID,ACQUISITIONCOSTS,EXPENSE_AMOUNT,Risk_Adjustment,CLAIMS_PAID_ADJ,TOTAL_COSTS,CONTRACT_COMBINED_RATIO_Adj,Profit_Code,Adjusted_CRatio,Profit_Code_Mod,LOB_UY_GROUP
0,91,2008,FIRE,FIRE_2008,448361.00,0.00,0.00,-17655.00,0.059,0.000000e+00,17655.00,1.352410,O,1.352410,O,FIRE_2008_O
1,91,2011,FIRE,FIRE_2011,1688514.00,-2155509.00,-629914.00,-15053.00,0.059,-2.282684e+06,2800476.00,1.352410,O,1.352410,O,FIRE_2011_O
2,91,2015,FIRE,FIRE_2015,2207111.00,-599126.00,-817207.00,-21128.00,0.059,-6.344744e+05,1437461.00,1.352410,O,1.352410,O,FIRE_2015_O
3,91,2015,FIRE,FIRE_2015,NaN,-1507410.75,0.00,0.00,0.059,-1.596348e+06,1507410.75,1.352410,O,1.352410,O,FIRE_2015_O
4,182,2007,FIRE,FIRE_2007,14039958.26,3449222.94,-4874103.52,0.00,0.059,3.652727e+06,1424880.58,0.688784,P,0.708784,P,FIRE_2007_P
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
120845,PTTY1911,2023,LIABILITY,LIABILITY_2023,18680.96,0.00,-5137.26,-10333.37,0.031,0.000000e+00,15470.63,0.463296,P,0.483296,P,LIABILITY_2023_P
120846,PTTY1921,2023,MISCELLANEOUS,MISCELLANEOUS_2023,NaN,0.00,0.00,0.00,0.112,0.000000e+00,-0.00,0.777536,P,0.797536,P,MISCELLANEOUS_2023_P
120847,PTTY1921,2023,MISCELLANEOUS,MISCELLANEOUS_2023,NaN,0.00,0.00,0.00,0.112,0.000000e+00,-0.00,0.777536,P,0.797536,P,MISCELLANEOUS_2023_P
120848,PTTY2130,2023,MARINE,MARINE_2023,22586.85,0.00,-6211.38,-225.87,0.084,0.000000e+00,6437.25,0.340414,P,0.360414,P,MARINE_2023_P


In [29]:
#In summary, this code performs group-wise aggregation on the 'TOTAL_COSTS' and 'PREMIUMS', 
#calculates a new column representing the ratio between these aggregated values, 
#and exports the resulting DataFrame to an Excel file for further analysis
lOB_PC = worked_LOB_Frame.groupby(['LINE_OF_BUSINESS','Profit_Code'])[['TOTAL_COSTS','PREMIUMS']] \
  .sum() \
  .assign(GROUP_COMBINED_RATIO = lambda x: x['TOTAL_COSTS'] / x['PREMIUMS'])
lOB_PC
#lOB_PC.to_excel('PivotTable_GroupRatio.xlsx')

TOTAL_COSTS      PREMIUMS  \
LINE_OF_BUSINESS      Profit_Code                               
AVIATION              O            2.286828e+07  1.573710e+07   
                      P            1.033752e+07  2.273718e+07   
                      U            3.498045e+05  3.975209e+05   
ENGINEERING           O            5.878129e+08  4.980359e+08   
                      P            1.547903e+09  2.559433e+09   
                      U            5.042166e+08  5.602954e+08   
FIRE                  O            8.742231e+09  6.789158e+09   
                      P            5.358994e+09  9.063334e+09   
                      U            2.075152e+09  2.269470e+09   
LIABILITY             O            3.851360e+07  2.405793e+07   
                      P            7.090562e+07  2.145662e+08   
                      U            5.451100e+04  6.000000e+04   
MARINE                O            1.026247e+09  8.591495e+08   
                      P            1.213178e+09  1.981394e+09   
                      U            1.720102e+08  1.931004e+08   
MEDICAL               O            5.246454e+09  5.321579e+09   
                      P            2.953113e+08  5.454591e+08   
                      U            1.291568e+09  1.415894e+09   
MISCELLANEOUS         O            1.627124e+09  1.384437e+09   
                      P            2.358613e+09  4.282275e+09   
                      U            5.273455e+08  5.830370e+08   
MOTOR                 O            2.062941e+09  1.474636e+09   
                      P            9.104168e+08  2.184626e+09   
                      U            4.668037e+08  5.112763e+08   
PERSONAL_ACCIDENT     O            1.666024e+07  6.170742e+06   
                      P            3.857384e+08  6.059812e+08   
                      U            3.407012e+07  3.844264e+07   
THEFT                 O            5.586941e+06  5.701530e+05   
                      P            5.811505e+06  2.130667e+07   
                      U            3.157260e+05  3.413000e+05   
WHOLE_ACCOUNT         O            4.775938e+08  4.130978e+08   
                      P            3.359296e+08  6.754226e+08   
                      U            2.598378e+08  3.001195e+08   
WORKMENS_COMPENSATION P            1.687852e+07  4.510039e+07   

                                   GROUP_COMBINED_RATIO  
LINE_OF_BUSINESS      Profit_Code                        
AVIATION              O                        1.453144  
                      P                        0.454653  
                      U                        0.879965  
ENGINEERING           O                        1.180262  
                      P                        0.604783  
                      U                        0.899912  
FIRE                  O                        1.287675  
                      P                        0.591283  
                      U                        0.914378  
LIABILITY             O                        1.600869  
                      P                        0.330460  
                      U                        0.908517  
MARINE                O                        1.194492  
                      P                        0.612285  
                      U                        0.890781  
MEDICAL               O                        0.985883  
                      P                        0.541399  
                      U                        0.912192  
MISCELLANEOUS         O                        1.175297  
                      P                        0.550785  
                      U                        0.904480  
MOTOR                 O                        1.398950  
                      P                        0.416738  
                      U                        0.913016  
PERSONAL_ACCIDENT     O                        2.699877  
                      P                        0.636552  
                      U                        0.886259  
THEFT        

In [30]:
premium_frame_base = worked_LOB_Frame.copy()
lOB_PC_COST = premium_frame_base.groupby(['LINE_OF_BUSINESS','Profit_Code'], as_index=False, group_keys=False)['TOTAL_COSTS'].sum()
lOB_PC_PREM = premium_frame_base.groupby(['LINE_OF_BUSINESS','Profit_Code'], as_index=False, group_keys=False)['PREMIUMS'].sum()
lOB_PC_COST['PREMIUMS'] = lOB_PC_PREM['PREMIUMS']
lOB_PC_COST['GROUP_COMBINED_RATIO'] = lOB_PC_COST['TOTAL_COSTS'] / lOB_PC_COST['PREMIUMS']
lOB_PC_COST['PROFIT_GROUP'] =  lOB_PC_COST['LINE_OF_BUSINESS']+'_'+lOB_PC_COST['Profit_Code']
lOB_PC_COST.to_excel('GROUP_COMBINED_RATIO_GROUPS.xlsx')
lOB_PC_COST
#lOB_PC_COST.to_excel('PivotTable27_Corect.xlsx')

,LINE_OF_BUSINESS,Profit_Code,TOTAL_COSTS,PREMIUMS,GROUP_COMBINED_RATIO,PROFIT_GROUP
0,AVIATION,O,2.286828e+07,1.573710e+07,1.453144,AVIATION_O
1,AVIATION,P,1.033752e+07,2.273718e+07,0.454653,AVIATION_P
2,AVIATION,U,3.498045e+05,3.975209e+05,0.879965,AVIATION_U
3,ENGINEERING,O,5.878129e+08,4.980359e+08,1.180262,ENGINEERING_O
4,ENGINEERING,P,1.547903e+09,2.559433e+09,0.604783,ENGINEERING_P
5,ENGINEERING,U,5.042166e+08,5.602954e+08,0.899912,ENGINEERING_U
6,FIRE,O,8.742231e+09,6.789158e+09,1.287675,FIRE_O
7,FIRE,P,5.358994e+09,9.063334e+09,0.591283,FIRE_P
8,FIRE,U,2.075152e+09,2.269470e+09,0.914378,FIRE_U
9,LIABILITY,O,3.851360e+07,2.405793e+07,1.600869,LIABILITY_O


# Creating a new column for LOB_UY_Group

In [31]:
worked_LOB_Frame
worked_LOB_Frame['UNDERWRITING_YEAR'] = worked_LOB_Frame['UNDERWRITING_YEAR'].astype('string')
worked_LOB_Frame['LOB_UY_GROUP'] = worked_LOB_Frame['POLICY_ID'] + '_' + worked_LOB_Frame['Profit_Code']
worked_LOB_Frame
#worked_LOB_Frame
worked_LOB_Frame['LOB_UY_GROUP'].drop_duplicates()

0                          FIRE_2008_O
1                          FIRE_2011_O
2                          FIRE_2015_O
4                          FIRE_2007_P
6                          FIRE_2006_P
                      ...             
106605    WORKMENS_COMPENSATION_2021_P
108299                    THEFT_2013_P
110370        PERSONAL_ACCIDENT_2022_O
115008                LIABILITY_2015_U
120660                    THEFT_2023_P
Name: LOB_UY_GROUP, Length: 599, dtype: string

In [32]:
worked_LOB_Frame

,CONTRACT_NO,UNDERWRITING_YEAR,LINE_OF_BUSINESS,POLICY_ID,PREMIUMS,CLAIM_PAID,ACQUISITIONCOSTS,EXPENSE_AMOUNT,Risk_Adjustment,CLAIMS_PAID_ADJ,TOTAL_COSTS,CONTRACT_COMBINED_RATIO_Adj,Profit_Code,Adjusted_CRatio,Profit_Code_Mod,LOB_UY_GROUP
0,91,2008,FIRE,FIRE_2008,448361.00,0.00,0.00,-17655.00,0.059,0.000000e+00,17655.00,1.352410,O,1.352410,O,FIRE_2008_O
1,91,2011,FIRE,FIRE_2011,1688514.00,-2155509.00,-629914.00,-15053.00,0.059,-2.282684e+06,2800476.00,1.352410,O,1.352410,O,FIRE_2011_O
2,91,2015,FIRE,FIRE_2015,2207111.00,-599126.00,-817207.00,-21128.00,0.059,-6.344744e+05,1437461.00,1.352410,O,1.352410,O,FIRE_2015_O
3,91,2015,FIRE,FIRE_2015,NaN,-1507410.75,0.00,0.00,0.059,-1.596348e+06,1507410.75,1.352410,O,1.352410,O,FIRE_2015_O
4,182,2007,FIRE,FIRE_2007,14039958.26,3449222.94,-4874103.52,0.00,0.059,3.652727e+06,1424880.58,0.688784,P,0.708784,P,FIRE_2007_P
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
120845,PTTY1911,2023,LIABILITY,LIABILITY_2023,18680.96,0.00,-5137.26,-10333.37,0.031,0.000000e+00,15470.63,0.463296,P,0.483296,P,LIABILITY_2023_P
120846,PTTY1921,2023,MISCELLANEOUS,MISCELLANEOUS_2023,NaN,0.00,0.00,0.00,0.112,0.000000e+00,-0.00,0.777536,P,0.797536,P,MISCELLANEOUS_2023_P
120847,PTTY1921,2023,MISCELLANEOUS,MISCELLANEOUS_2023,NaN,0.00,0.00,0.00,0.112,0.000000e+00,-0.00,0.777536,P,0.797536,P,MISCELLANEOUS_2023_P
120848,PTTY2130,2023,MARINE,MARINE_2023,22586.85,0.00,-6211.38,-225.87,0.084,0.000000e+00,6437.25,0.340414,P,0.360414,P,MARINE_2023_P


## 5. Calculating the premium ratio

* grouping by Line of business, underwriting year and profit code then aggregating the costs and premiums 
* Its total grouped costs over similarily grouped premiums 

In [33]:
prem_v2 = worked_LOB_Frame.copy()
prem_top = prem_v2.groupby(['LINE_OF_BUSINESS','Profit_Code','UNDERWRITING_YEAR'], as_index=False, group_keys=False)['PREMIUMS'].sum()
prem_bot = prem_v2.groupby(['LINE_OF_BUSINESS','UNDERWRITING_YEAR'], as_index=False, group_keys=False)['PREMIUMS'].sum()
prem = pd.merge(prem_top,prem_bot,on=['LINE_OF_BUSINESS','UNDERWRITING_YEAR'], how='inner')
prem['Premium_Ratio'] = prem['PREMIUMS_x'] / prem['PREMIUMS_y']
prem.drop(columns=['PREMIUMS_x', 'PREMIUMS_y'], axis = 1, inplace=True) 
#Saving premium ratios to excel
#prem.to_excel('Premium_Ratios.xlsx')
prem

,LINE_OF_BUSINESS,Profit_Code,UNDERWRITING_YEAR,Premium_Ratio
0,AVIATION,O,2013,0.766800
1,AVIATION,P,2013,0.233200
2,AVIATION,O,2014,0.622368
3,AVIATION,P,2014,0.377632
4,AVIATION,U,2014,0.000000
...,...,...,...,...
594,WORKMENS_COMPENSATION,P,2015,1.000000
595,WORKMENS_COMPENSATION,P,2021,NaN
596,WORKMENS_COMPENSATION,P,2022,1.000000
597,WORKMENS_COMPENSATION,P,2023,1.000000


# * End of section. 

In [34]:
#End of section 
end = timer()
duration = end - start3
print('The took calculations ' + str(duration) + '  seconds to run the transformations')

#End of Transformations 
end = timer()
duration = end - start
print('The program has completed sucessfully and outputed 4 computed sheets, it took ' + str(duration) + '  seconds to run and save the files')
print('Thank you for your patience :) ')

The took calculations 1.7537493999989238  seconds to run the transformations
The program has completed sucessfully and outputed 4 computed sheets, it took 7.757796999998391  seconds to run and save the files
Thank you for your patience :) 


# Program has finished running the first section and will output 4 files

The next section includes the acturial input. We are working on the...

 
#                                         Profitability Groups

* Use this LOB_UY_Group to group - call it profitability grouping 
* Group by LOB AND UY, add everything together, group them by P,O,U , get combined ratio  and combined adjusted ratio 
* Leave a way to add a Risk adjustment 


In [35]:
#Creating a dataframe to combine with act input, we only need the first column
grouped_LOB = worked_LOB_Frame['LOB_UY_GROUP']
#Dropped duplicates to be left with the first instance of each     
grouped_LOB.drop_duplicates(inplace=True)
#Creating it
grouped_LOB =pd.DataFrame(grouped_LOB)
#Splitting the column into components
grouped_LOB['LOB'] = grouped_LOB['LOB_UY_GROUP'].str[:-7]
grouped_LOB['UY'] = grouped_LOB['LOB_UY_GROUP'].str[-6:-2]
grouped_LOB['PC'] = grouped_LOB['LOB_UY_GROUP'].str[-1]

#grouped_LOB
#worked_LOB_Frame


In [36]:
#We will then aggregate our numerical information into this abstracted level of grouping to match with the unique values
grouped_LOB['GROUPED_PREMIUM_PG'] = worked_LOB_Frame.groupby(['LINE_OF_BUSINESS','Profit_Code_Mod'])['PREMIUMS'].transform('sum')
grouped_LOB['GROUPED_CLAIMS_PG'] = worked_LOB_Frame.groupby(['LINE_OF_BUSINESS','Profit_Code_Mod'])['CLAIMS_PAID_ADJ'].transform('sum')
grouped_LOB['GROUPED_AQCST_PG'] = worked_LOB_Frame.groupby(['LINE_OF_BUSINESS','Profit_Code_Mod'])['ACQUISITIONCOSTS'].transform('sum')
grouped_LOB['GROUPED_EXPENSE_PG'] = worked_LOB_Frame.groupby(['LINE_OF_BUSINESS','Profit_Code_Mod'])['EXPENSE_AMOUNT'].transform('sum')

#Creating a sumed cost column
grouped_LOB['GROUPED_OUTGO_PG'] = grouped_LOB['GROUPED_CLAIMS_PG']+grouped_LOB['GROUPED_AQCST_PG']+grouped_LOB['GROUPED_EXPENSE_PG']
grouped_LOB['GROUPED_OUTGO_PG'] = grouped_LOB['GROUPED_OUTGO_PG'] *-1

#Calcualting the ratio, again  
grouped_LOB['COMBINED_RATIO_PG'] = grouped_LOB['GROUPED_OUTGO_PG'] / grouped_LOB['GROUPED_PREMIUM_PG']
#expense_loading = 0.05 # Adding expense loading
grouped_LOB['COMBINED_RATIO_PG'] = grouped_LOB['COMBINED_RATIO_PG'] + expense_loading

#printing to excel 
#grouped_LOB.to_excel('Grouped_Combined_Ratio_27_12.xlsx')
#

In [37]:
expense_loading

0.055414

In [38]:
grouped_LOB

,LOB_UY_GROUP,LOB,UY,PC,GROUPED_PREMIUM_PG,GROUPED_CLAIMS_PG,GROUPED_AQCST_PG,GROUPED_EXPENSE_PG,GROUPED_OUTGO_PG,COMBINED_RATIO_PG
0,FIRE_2008_O,FIRE,2008,O,6.789158e+09,-6.796281e+09,-2.139833e+09,-1.847569e+08,9.120872e+09,1.398861
1,FIRE_2011_O,FIRE,2011,O,6.789158e+09,-6.796281e+09,-2.139833e+09,-1.847569e+08,9.120872e+09,1.398861
2,FIRE_2015_O,FIRE,2015,O,6.789158e+09,-6.796281e+09,-2.139833e+09,-1.847569e+08,9.120872e+09,1.398861
4,FIRE_2007_P,FIRE,2007,P,8.025574e+09,-1.998965e+09,-2.325148e+09,-2.856430e+08,4.609757e+09,0.629797
6,FIRE_2006_P,FIRE,2006,P,8.025574e+09,-1.998965e+09,-2.325148e+09,-2.856430e+08,4.609757e+09,0.629797
...,...,...,...,...,...,...,...,...,...,...
106605,WORKMENS_COMPENSATION_2021_P,WORKMENS_COMPENSATION,2021,P,4.510039e+07,-5.519844e+06,-1.135113e+07,-7.543930e+03,1.687852e+07,0.429657
108299,THEFT_2013_P,THEFT,2013,P,2.130667e+07,-1.307075e+06,-4.504430e+06,0.000000e+00,5.811505e+06,0.328169
110370,PERSONAL_ACCIDENT_2022_O,PERSONAL_ACCIDENT,2022,O,6.170742e+06,-1.601658e+07,-1.100765e+06,-2.448578e+04,1.714183e+07,2.833334
115008,LIABILITY_2015_U,LIABILITY,2015,U,6.000000e+04,-4.073584e+04,-1.500000e+04,0.000000e+00,5.573584e+04,0.984345


## Onto Premium Ratios

In [39]:
#Creating a copy of the dataset
prem_ratios = grouped_LOB.copy()

#Focusing on the columns we need
prem_ratios = prem_ratios[['LOB_UY_GROUP','LOB','UY','PC','GROUPED_PREMIUM_PG','GROUPED_CLAIMS_PG','GROUPED_AQCST_PG','GROUPED_EXPENSE_PG','GROUPED_OUTGO_PG']]
#Calculating the numerator diff by the Modififed Profit Code
prem_ratios['PREM_BY_UY_LOB_PC'] = worked_LOB_Frame.groupby(['LINE_OF_BUSINESS','Profit_Code_Mod','UNDERWRITING_YEAR'])['PREMIUMS'].transform('sum')
prem_ratios['PREM_BY_UY_LOB'] = worked_LOB_Frame.groupby(['LINE_OF_BUSINESS','UNDERWRITING_YEAR'])['PREMIUMS'].transform('sum')
#Calculating the premium ratio
prem_ratios['PREM_RATIO'] = prem_ratios['PREM_BY_UY_LOB_PC'] / prem_ratios['PREM_BY_UY_LOB']
#prem_ratios

# 

In [40]:
#Preprocessing of ACT input
act_input['Class of Business'] = act_input['Class of Business'].str.upper()
act_input.info()
act_input['Class of Business'].value_counts()
#Modifiying the columns
act_input = act_input.rename(columns={'Class of Business': 'LOB'}) 
act_input['LOB'] = act_input['LOB'].astype('string')


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 162 entries, 0 to 161
Data columns (total 19 columns):
 #   Column               Non-Null Count  Dtype         
---  ------               --------------  -----         
 0   Class of Business    162 non-null    object        
 1   UY                   162 non-null    int64         
 2   As at                162 non-null    datetime64[ns]
 3   IBNR/LIC             162 non-null    float64       
 4   Pipeline Premium     162 non-null    float64       
 5   Pipeline Commission  162 non-null    float64       
 6   Booked UPR           162 non-null    float64       
 7   Pipeline UPR         162 non-null    float64       
 8   Booked DAC           162 non-null    float64       
 9   Pipeline DAC         162 non-null    float64       
 10  IFIE                 162 non-null    float64       
 11  Risk Adjustment      162 non-null    float64       
 12  Pipleine PC          162 non-null    float64       
 13  Pipeline Claims      162 non-null  

In [41]:
#Well know filter the dataframe using the same reporting year details
print('This is the reporting date we will use: ',reporting_date)
#We now filter our data set using the dates we have 
act_input['As_at'] = act_input['As at'].dt.date
#Filter by year and Quater
act_input = act_input.loc[act_input['As_at'] == reporting_date]#By the FINANCIAL quaater
act_input = act_input.loc[act_input['UY'] <= year]
act_input['UY'] = act_input['UY'].astype('string')
act_input.info()#Taking a look at what we are left with 

This is the reporting date we will use:  2024-09-30
<class 'pandas.core.frame.DataFrame'>
Index: 162 entries, 0 to 161
Data columns (total 20 columns):
 #   Column               Non-Null Count  Dtype         
---  ------               --------------  -----         
 0   LOB                  162 non-null    string        
 1   UY                   162 non-null    string        
 2   As at                162 non-null    datetime64[ns]
 3   IBNR/LIC             162 non-null    float64       
 4   Pipeline Premium     162 non-null    float64       
 5   Pipeline Commission  162 non-null    float64       
 6   Booked UPR           162 non-null    float64       
 7   Pipeline UPR         162 non-null    float64       
 8   Booked DAC           162 non-null    float64       
 9   Pipeline DAC         162 non-null    float64       
 10  IFIE                 162 non-null    float64       
 11  Risk Adjustment      162 non-null    float64       
 12  Pipleine PC          162 non-null    float64 

In [42]:
#Creating a slice to work with
act_input_edited = act_input[['LOB','UY','Total UPR','Total DAC']]
print(act_input_edited.info())
#A copy of the previous output
working_table = prem_ratios.copy()
#print(working_table.info())

<class 'pandas.core.frame.DataFrame'>
Index: 162 entries, 0 to 161
Data columns (total 4 columns):
 #   Column     Non-Null Count  Dtype  
---  ------     --------------  -----  
 0   LOB        162 non-null    string 
 1   UY         162 non-null    string 
 2   Total UPR  162 non-null    float64
 3   Total DAC  162 non-null    float64
dtypes: float64(2), string(2)
memory usage: 6.3 KB
None


In [43]:
# Merge df1 and df2 based on 'ID' column
merged_df = pd.merge(working_table, act_input_edited, on=['LOB','UY'], how='left')  # Merge based on 'ID', keeping all rows from df1

merged_df = merged_df.fillna(0)
merged_df.head()
# # Create a new column 'New_Column' in df1 and fill values based on conditions from df2
merged_df['Total UPR'] = merged_df['Total UPR'] * merged_df['PREM_RATIO']
merged_df['Total DAC'] = merged_df['Total DAC'] * merged_df['PREM_RATIO']
working_table['LOB'].value_counts()
#act_input_edited['LOB'].value_counts()

LOB
MISCELLANEOUS            72
FIRE                     71
ENGINEERING              71
MARINE                   70
MOTOR                    70
PERSONAL_ACCIDENT        49
MEDICAL                  45
AVIATION                 42
LIABILITY                41
WHOLE_ACCOUNT            40
THEFT                    18
WORKMENS_COMPENSATION    10
Name: count, dtype: Int64

We need to include the combined ratio

In [44]:
cond = merged_df['LOB_UY_GROUP'].unique()

frank_1 = grouped_LOB[grouped_LOB['LOB_UY_GROUP'].isin(cond)]

# Assuming 'LOB_UY_GROUP' in frank_1 matches 'LOB_UY_GROUP' in merged_df
merged_df['COMBINED_RATIO'] = frank_1.loc[frank_1['LOB_UY_GROUP'].isin(cond), 'COMBINED_RATIO_PG']

#Joining and Transforming
frank = frank_1[['LOB_UY_GROUP','COMBINED_RATIO_PG']]
merged_df = pd.merge(merged_df, frank, on='LOB_UY_GROUP', how='inner')

merged_df['LRC'] = (merged_df['Total UPR'] - merged_df['Total DAC']) 


merged_df.drop(['COMBINED_RATIO'], axis=1, inplace=True)

merged_df.rename(columns ={'COMBINED_RATIO_PG_y':'COMBINED_RATIO_PG'}, inplace = True)
merged_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 599 entries, 0 to 598
Data columns (total 16 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   LOB_UY_GROUP        599 non-null    string 
 1   LOB                 599 non-null    string 
 2   UY                  599 non-null    string 
 3   PC                  599 non-null    string 
 4   GROUPED_PREMIUM_PG  599 non-null    float64
 5   GROUPED_CLAIMS_PG   599 non-null    float64
 6   GROUPED_AQCST_PG    599 non-null    float64
 7   GROUPED_EXPENSE_PG  599 non-null    float64
 8   GROUPED_OUTGO_PG    599 non-null    float64
 9   PREM_BY_UY_LOB_PC   599 non-null    float64
 10  PREM_BY_UY_LOB      599 non-null    float64
 11  PREM_RATIO          599 non-null    float64
 12  Total UPR           597 non-null    float64
 13  Total DAC           597 non-null    float64
 14  COMBINED_RATIO_PG   599 non-null    float64
 15  LRC                 597 non-null    float64
dtypes: float

### PVFCF Calculation Check

* The formula should only be applied to 'Onerous' contracts 

In [45]:
# Value to identify rows for modification
target_value = 'O'

# Increment values in Column_B for rows where Column_A equals target_value
merged_df['PVFCF'] = merged_df.loc[merged_df['PC'] == target_value, 'LRC'] * merged_df['COMBINED_RATIO_PG']

In [46]:
# Value to identify rows for modification
target_value = 'O'

# Multiply values in 'PlaceHolder' with 'COMBINED_RATIO_PG' only for rows where 'PC' equals target_value
merged_df.loc[merged_df['PC'] == target_value, 'PVFCF'] = merged_df.loc[merged_df['PC'] == target_value, 'LRC'] * merged_df.loc[merged_df['PC'] == target_value, 'COMBINED_RATIO_PG']

merged_df.head(5)

#merged_df['PVFCF']= merged_df['PlaceHolder']* merged_df['COMBINED_RATIO_PG']

# #Next is the Loss Component
merged_df['Loss_Component'] = np.maximum(merged_df['PVFCF']- (merged_df['Total UPR'] - merged_df['Total DAC']), 0)
merged_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 599 entries, 0 to 598
Data columns (total 18 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   LOB_UY_GROUP        599 non-null    string 
 1   LOB                 599 non-null    string 
 2   UY                  599 non-null    string 
 3   PC                  599 non-null    string 
 4   GROUPED_PREMIUM_PG  599 non-null    float64
 5   GROUPED_CLAIMS_PG   599 non-null    float64
 6   GROUPED_AQCST_PG    599 non-null    float64
 7   GROUPED_EXPENSE_PG  599 non-null    float64
 8   GROUPED_OUTGO_PG    599 non-null    float64
 9   PREM_BY_UY_LOB_PC   599 non-null    float64
 10  PREM_BY_UY_LOB      599 non-null    float64
 11  PREM_RATIO          599 non-null    float64
 12  Total UPR           597 non-null    float64
 13  Total DAC           597 non-null    float64
 14  COMBINED_RATIO_PG   599 non-null    float64
 15  LRC                 597 non-null    float64
 16  PVFCF   

In [47]:
merged_df['Loss_Component'].value_counts()

Loss_Component
0.000000e+00    171
2.245417e+06      1
1.333514e+03      1
6.329668e+06      1
1.740450e+04      1
2.008835e+06      1
2.245552e+04      1
5.943403e+06      1
2.634164e+03      1
2.063633e+03      1
1.052285e+07      1
9.040743e+04      1
5.629405e+03      1
1.209463e+03      1
3.807176e+07      1
6.635886e+05      1
2.781117e+04      1
1.580539e+05      1
6.295789e+05      1
Name: count, dtype: int64

In [48]:
#Printing to excel
merged_df.to_excel('NL_Loss_Component_Q3_2024.xlsx')

In [49]:
merged_df['Loss_Component'].sum()

66744098.37797358

In [50]:
print(f"This is the year: ",year,month,day)

This is the year:  2024 9 30
